<a href="https://colab.research.google.com/github/dipikachavan06/data_science_lab/blob/main/experiment%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### A* (A-star) Search Algorithm

A* is a popular pathfinding algorithm, widely used in various applications like games, robotics, and logistics. It is an informed search algorithm, meaning it uses a heuristic function to guide its search, making it more efficient than uninformed search algorithms like BFS or DFS for finding the shortest path.

**How A* Works:**

A* works by maintaining two lists:
1.  **Open List (Frontier):** Contains nodes that have been visited but whose neighbors haven't all been explored yet.
2.  **Closed List:** Contains nodes that have been fully evaluated.

For each node, A* calculates a cost function `f(n) = g(n) + h(n)`:
*   `g(n)`: The actual cost from the start node to the current node `n`.
*   `h(n)`: The estimated cost (heuristic) from the current node `n` to the end node. This heuristic must be admissible (never overestimates the cost).

The algorithm iteratively expands the node with the lowest `f(n)` value from the open list until the goal node is reached or the open list is empty.

In [5]:
import heapq

def a_star(grid, start, end):
    # Define possible movements (up, down, left, right, diagonals)
    # For a simple grid, we'll consider 4 directions initially
    # dx = [-1, 1, 0, 0]
    # dy = [0, 0, -1, 1]

    # For 8 directions (including diagonals)
    dx = [-1, -1, -1, 0, 0, 1, 1, 1]
    dy = [-1, 0, 1, -1, 1, -1, 0, 1]

    rows, cols = len(grid), len(grid[0])

    # g_score: cost from start to current node
    g_score = { (r, c): float('inf') for r in range(rows) for c in range(cols) }
    g_score[start] = 0

    # f_score: g_score + heuristic (estimated cost from current to end)
    f_score = { (r, c): float('inf') for r in range(rows) for c in range(cols) }

    # Heuristic: Manhattan distance or Euclidean distance
    # Manhattan distance is often used for 4-directional movement
    # Euclidean distance is often used for 8-directional movement
    def heuristic(node, target):
        return ((node[0] - target[0])**2 + (node[1] - target[1])**2)**0.5 # Euclidean
        # return abs(node[0] - target[0]) + abs(node[1] - target[1]) # Manhattan

    f_score[start] = heuristic(start, end)

    # Priority queue to store nodes to be evaluated (f_score, node)
    open_list = [(f_score[start], start)] # (f_score, (row, col))

    # came_from: reconstructs the path
    came_from = {}

    while open_list:
        current_f, current_node = heapq.heappop(open_list)

        if current_node == end:
            # Reconstruct path
            path = []
            while current_node in came_from:
                path.append(current_node)
                current_node = came_from[current_node]
            path.append(start)
            return path[::-1]

        for i in range(len(dx)):
            neighbor_r, neighbor_c = current_node[0] + dx[i], current_node[1] + dy[i]
            neighbor = (neighbor_r, neighbor_c)

            # Check bounds and if it's an obstacle (1 in grid means obstacle)
            if 0 <= neighbor_r < rows and 0 <= neighbor_c < cols and grid[neighbor_r][neighbor_c] == 0:
                # Calculate tentative g_score for the neighbor
                # Cost of moving to an adjacent node (1 for horizontal/vertical, sqrt(2) for diagonal)
                if dx[i] == 0 or dy[i] == 0:
                    # Straight movement
                    tentative_g_score = g_score[current_node] + 1
                else:
                    # Diagonal movement
                    tentative_g_score = g_score[current_node] + (2**0.5)

                if tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current_node
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = g_score[neighbor] + heuristic(neighbor, end)
                    if (f_score[neighbor], neighbor) not in open_list:
                        heapq.heappush(open_list, (f_score[neighbor], neighbor))

    return None # No path found


### Minimax Algorithm

Minimax is a decision rule used in artificial intelligence, decision theory, game theory, statistics, and philosophy for minimizing the possible loss for a worst-case (maximum loss) scenario. It is often used in two-player games like Tic-Tac-Toe, Chess, and Checkers.

**How Minimax Works:**

The algorithm works by recursively exploring all possible moves from the current game state until a terminal state (win, loss, or draw) or a predefined depth limit is reached. It assumes both players play optimally.

*   **Maximizing Player:** Aims to maximize their score.
*   **Minimizing Player:** Aims to minimize the opponent's (and thus their own) score.

The algorithm assigns a value to each terminal state (e.g., +1 for a win, -1 for a loss, 0 for a draw). For non-terminal states, it propagates these values back up the game tree:

*   **For the Maximizing Player's turn:** Choose the move that leads to the state with the maximum value returned by the recursive call.
*   **For the Minimizing Player's turn:** Choose the move that leads to the state with the minimum value returned by the recursive call.

In [6]:
def minimax(board, depth, is_maximizing_player):
    # Base cases: Check if game is over or depth limit reached
    score = evaluate(board)
    if score != 0: # Game is won/lost
        return score
    if is_game_over(board): # Game is a draw
        return 0
    if depth == 0: # Depth limit reached
        return 0 # Or a heuristic evaluation if not a terminal state

    if is_maximizing_player:
        best_score = -float('inf')
        for move in get_available_moves(board):
            new_board = make_move(board, move, 'X') # Assume 'X' is maximizing player
            score = minimax(new_board, depth - 1, False)
            best_score = max(best_score, score)
        return best_score
    else:
        best_score = float('inf')
        for move in get_available_moves(board):
            new_board = make_move(board, move, 'O') # Assume 'O' is minimizing player
            score = minimax(new_board, depth - 1, True)
            best_score = min(best_score, score)
        return best_score


def find_best_move(board, depth, is_maximizing_player):
    best_score = -float('inf') if is_maximizing_player else float('inf')
    best_move = None

    for move in get_available_moves(board):
        player_symbol = 'X' if is_maximizing_player else 'O'
        new_board = make_move(board, move, player_symbol)
        score = minimax(new_board, depth - 1, not is_maximizing_player)

        if is_maximizing_player and score > best_score:
            best_score = score
            best_move = move
        elif not is_maximizing_player and score < best_score:
            best_score = score
            best_move = move
    return best_move

# --- Helper functions for a simple Tic-Tac-Toe example ---
# (These would be specific to your game)

def create_board():
    return [[' ' for _ in range(3)] for _ in range(3)]

def print_board(board):
    for row in board:
        print('|' + '|'.join(row) + '|')
    print()

def get_available_moves(board):
    moves = []
    for r in range(3):
        for c in range(3):
            if board[r][c] == ' ':
                moves.append((r, c))
    return moves

def make_move(board, move, player):
    new_board = [row[:] for row in board] # Create a deep copy
    r, c = move
    new_board[r][c] = player
    return new_board

def check_win(board, player):
    # Check rows, columns, and diagonals
    for i in range(3):
        if all(board[i][j] == player for j in range(3)): return True # Rows
        if all(board[j][i] == player for j in range(3)): return True # Columns
    if all(board[i][i] == player for i in range(3)): return True # Diagonal \\
    if all(board[i][2-i] == player for i in range(3)): return True # Diagonal /
    return False

def is_game_over(board):
    return check_win(board, 'X') or check_win(board, 'O') or not get_available_moves(board)

def evaluate(board):
    if check_win(board, 'X'):
        return 1 # Maximizing player wins
    elif check_win(board, 'O'):
        return -1 # Minimizing player wins
    else:
        return 0 # Draw or game not over


### Example Usage (Tic-Tac-Toe)

In [7]:
# Initial empty Tic-Tac-Toe board
board = create_board()
print("Initial Board:")
print_board(board)

# Example 1: Maximizing player ('X') to move, aiming to win
# Let's set up a board where 'X' can win in one move
board_scenario1 = [
    ['X', 'O', ' '],
    [' ', 'X', ' '],
    [' ', ' ', 'O']
]
print("Scenario 1: Maximizing player 'X' to move and win")
print_board(board_scenario1)

best_move_x = find_best_move(board_scenario1, depth=2, is_maximizing_player=True)
print(f"Best move for 'X': {best_move_x}")
if best_move_x:
    board_scenario1_after_move = make_move(board_scenario1, best_move_x, 'X')
    print("Board after X's optimal move:")
    print_board(board_scenario1_after_move)

# Example 2: Minimizing player ('O') to move, trying to block 'X'
board_scenario2 = [
    ['X', 'O', ' '],
    [' ', ' ', ' '],
    ['X', ' ', 'O']
]
print("Scenario 2: Minimizing player 'O' to move and block 'X'")
print_board(board_scenario2)

best_move_o = find_best_move(board_scenario2, depth=2, is_maximizing_player=False)
print(f"Best move for 'O': {best_move_o}")
if best_move_o:
    board_scenario2_after_move = make_move(board_scenario2, best_move_o, 'O')
    print("Board after O's optimal move:")
    print_board(board_scenario2_after_move)


Initial Board:
| | | |
| | | |
| | | |

Scenario 1: Maximizing player 'X' to move and win
|X|O| |
| |X| |
| | |O|

Best move for 'X': (0, 2)
Board after X's optimal move:
|X|O|X|
| |X| |
| | |O|

Scenario 2: Minimizing player 'O' to move and block 'X'
|X|O| |
| | | |
|X| |O|

Best move for 'O': (1, 0)
Board after O's optimal move:
|X|O| |
|O| | |
|X| |O|



### Example Usage

In [4]:
# Example Grid: 0 for traversable, 1 for obstacle
grid = [
    [0, 0, 0, 0, 0],
    [0, 1, 1, 0, 0],
    [0, 0, 0, 1, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0]
]

start_node = (0, 0)  # Start at top-left
end_node = (4, 4)    # End at bottom-right

path = a_star(grid, start_node, end_node)

if path:
    print(f"Path from {start_node} to {end_node}: {path}")

    # Visualize the path on the grid
    display_grid = [list(row) for row in grid] # Create a copy to modify
    for r, c in path:
        if (r, c) != start_node and (r, c) != end_node:
            display_grid[r][c] = '*'

    print("\nGrid with path:")
    for row in display_grid:
        print(' '.join(map(str, row)))

else:
    print(f"No path found from {start_node} to {end_node}.")


# Another example with an impossible path
grid_impossible = [
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
]

start_node_impossible = (0, 0)
end_node_impossible = (4, 4)

path_impossible = a_star(grid_impossible, start_node_impossible, end_node_impossible)

if path_impossible:
    print(f"\nPath for impossible grid: {path_impossible}")
else:
    print(f"\nNo path found for impossible grid from {start_node_impossible} to {end_node_impossible}.")

Path from (0, 0) to (4, 4): [(0, 0), (1, 0), (2, 1), (3, 2), (3, 3), (4, 4)]

Grid with path:
0 0 0 0 0
* 1 1 0 0
0 * 0 1 0
0 1 * * 0
0 0 0 0 0

Path for impossible grid: [(0, 0), (0, 1), (0, 2), (0, 3), (1, 4), (2, 4), (3, 4), (4, 4)]


### Breadth-First Search (BFS)

BFS is an algorithm for traversing or searching tree or graph data structures. It starts at the tree root (or some arbitrary node of a graph, sometimes referred to as a 'search key') and explores all of the neighbor nodes at the present depth prior to moving on to the nodes at the next depth level.

In [1]:
from collections import deque

def bfs(graph, start_node):
    visited = set()
    queue = deque([start_node])
    traversal_order = []

    visited.add(start_node)

    while queue:
        current_node = queue.popleft()
        traversal_order.append(current_node)

        for neighbor in graph[current_node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)

    return traversal_order

# Example Graph
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

print("BFS Traversal starting from 'A':", bfs(graph, 'A'))

BFS Traversal starting from 'A': ['A', 'B', 'C', 'D', 'E', 'F']


### Depth-First Search (DFS)

DFS is an algorithm for traversing or searching tree or graph data structures. The algorithm starts at the root node (selecting some arbitrary node as the root node in the case of a graph) and explores as far as possible along each branch before backtracking.

In [2]:
def dfs(graph, start_node, visited=None, traversal_order=None):
    if visited is None:
        visited = set()
    if traversal_order is None:
        traversal_order = []

    visited.add(start_node)
    traversal_order.append(start_node)

    for neighbor in graph[start_node]:
        if neighbor not in visited:
            dfs(graph, neighbor, visited, traversal_order)

    return traversal_order

# Example Graph (same as BFS example)
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

print("DFS Traversal starting from 'A':", dfs(graph, 'A'))

DFS Traversal starting from 'A': ['A', 'B', 'D', 'E', 'F', 'C']
